# Section 1:  Data Analysis and Pre-Processing

## Introduction to short course data and problem

The goal of the problem is to predict the probability of low-level vorticity exceeding a certain threshold up to ahead given a storm with a simulated radar reflectivity > 40 dBZ and the associated surface wind and temperature fields. 

__Input fields in the netCDF data:__

* REFL_COM_curr (composite reflectivity)

* U10_curr (10 m west-east wind component in m/s)

* V10_curr (10 m south-north wind component in m/s)

* T2_curr (2 m temperature in Kelvin)

__Prediction field:__

* RVORT1_MAX_future (hourly maximum vertical vorticity at 1 km Above ground level in s-1)

__Other fields of note:__

* time: valid time of storm image

* i and j: row and column array indices from original WRF model grid

* x and y: projection coordinates in m

* masks: binary grid showing where storm outline is. Aggregate stats in csv file are extracted from only positive grid points in masks.

## Reading meteorological data files with pandas and xarray

First we need to import the libraries used in this section

In [ ]:
%matplotlib inline
import numpy as np
import netCDF4 as nc
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from scipy.stats import percentileofscore

import glob
import os

##### How to find the CSV files and create a sorted list of the found files
To do this, we use the glob library to list all of the *.csv files in the specified directory.

In [ ]:
# set the path to the data as a string
path = "../../../data/track_data_ncar_ams_3km_csv_small/"

# create a list of the files and print it out
files = sorted(glob.glob(path+"/*.csv"))

##### How to read in all of the found CSV files using Pandas and concat all of the content
This method adds the content of all of the csv files into one Python Pandas DataFrame object.  We also print the data's column labels in order to help us determine which keys we can use. 

In [ ]:
df = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)
print (df.keys())

### Exploring the CSV data
It's always important to understand your data before you do anything with it!  If you don't understand your data, your analysis will be difficult and conclusions can be incorrect.


##### First lets get a subsection of the data into a DataFrame object by using the labels found by printing the list of keys.

In [ ]:
df1 = df.loc[:,['Step_ID', 'U10_mean', 'V10_mean', 'T2_mean']]
print (type(df1))

##### Once we have the data within a DataFrame, it's easy to get the mean by issueing this command.

In [ ]:
df1['T2_mean'].mean()

##### It's also easy to create a quick plot

In [ ]:
df1['T2_mean'].plot()

### Exploring NetCDF files

##### How to find the NetCDF files and create a sorted list of the found files
To do this, we use the glob library to list all of the *.nc files in the specified directory.

In [ ]:
# set the path to the data as a string
path = "../../../data/track_data_ncar_ams_3km_nc_small/"

# create a list of the files and print it out
files = sorted(glob.glob(path+"/*.nc"))
#print (files)

 ##### Let's read in the first file in the list and see what is in the files.
 
 In the below cell, we open the first file within the file list and print out its summary information.  Afterwards, we close the file.

In [ ]:
#  Open the file with the Netcdf library for reading and then close it
nf = nc.Dataset(files[0], "r")
print (nf)
nf.close()

##### Now lets open a file with xarray and print its content

As you probably have noticed, the output format is easier to read.

In [ ]:
xf = xr.open_dataset(files[0])
print (xf)

##### Here are a few different ways to look at the different parts of a NetCDF file using xarray

You can reference variables as keys within the xarray object.

You can use the variable's attributes to reference the dimensions, coordinates, and attributes of that variable.

In [ ]:
T = xf['T2_curr']
print (T)
print ("Dimensions:",T.dims)
print ("Coords:",T.coords)
print ("Attributes:",T.attrs)

##### Indexing and Selecting Data Ranges

There are difference ways to retrieve the values of a variable. 

You can use indexing similar to numpy arrays.  You can also index using diminsion names.

In [ ]:
print (T[0,0,1].values)
print (T[0:5,0,1].values)
print (T.sel(p=slice(0,5),col=1,row=0).values)
print (T.sel(p=4,col=1,row=0).values)
print (T.sel(row=0,col=1,p=4).values)

## Data transformations
For these examples, we will use the xarray object

##### Create an array where the values are larger than 290 and values less than 290 are added as nan values.

In [ ]:
T.where(T>290)

##### Create an array where values greater than 290 are True and values less than 290 are False

In [ ]:
T>290

##### Round all values to the nearest integer

In [ ]:
T.round(0)

##### Find the mean of all the values for that variable

In [ ]:
T.mean(dim=['col','row','p']).values


##### Find the min of all values for that variable and then find the min across all columns and rows

In [ ]:
print (T.min())
print (T.min(dim=['col','row']))

##### Find the max of all values for that variable and then find the min across all columns and rows

In [ ]:
print (T.max())
print (T.max(dim=['col','row']))

##### Compute the 5th percentile of the data along the 'p' axis.

In [ ]:
print (T.groupby('p').reduce(np.percentile, q=5))

## Exploratory visulization with matplotlib 
As with anything you do with Python, there are multiple ways of doing the same thing.  Here are a couple of ways to create plots.

##### How to create a simple line plot

In [ ]:
T[:,1,1].plot()

##### How to create a simple plot of p=0

In [ ]:
T.isel(p=0).plot()

##### How to create a timeseries plot over two locations using matplotlib

In [ ]:
plt.plot(xf['p'],T[:,1,1], label='Location 1')
plt.plot(xf['p'],T[:,30,30], label='Location 2')
plt.ylabel('Temperature (K)')
plt.xlabel('Time')
plt.legend()

##### How to create a simple contour plot with matplotlib

In [ ]:
plt.contour(T[0,:,:])

#####  Create the same plot as above, but use the axis label provided by xarray

In [ ]:
plt.contour(T.sel(p=0))


##### Same as above, but countour labels have been added

In [ ]:
cs = plt.contour(T.sel(p=0))
plt.clabel(cs, fmt='%.0f', inline=True)

##### Set some variables that are used in the following examples

In [ ]:
V = xf['V10_curr']
U = xf['U10_curr']
r = xf['row']
c = xf['col']

##### How to draw a countour plot with quivers

In [ ]:
cs = plt.contour(U.sel(p=0))
plt.clabel(cs, fmt='%.0f', inline=True)
plt.quiver(r, c, U.sel(p=0), V.sel(p=0), pivot='middle')

##### How to draw a barb plot

In [ ]:
plt.barbs(r, c, U.sel(p=200), V.sel(p=200), length=5, pivot='middle')

# Preparing data for the turorial

##### Declare all of the input and output variables

In [ ]:
run_times = []
valid_times = []
# List of input variables
in_vars = ["REFL_COM_curr",
           "U10_curr", "V10_curr"]
# List of output variables
out_vars = ["RVORT1_MAX_future"]
in_data = []
out_data = []

##### Loop through the first 5 files and extract the relevant variables
We're only operating on a couple of files for the following example to save on memory

In [ ]:
for f in files[0:5]:
    run_time = pd.Timestamp(f.split("/")[-1].split("_")[1])
    ds = xr.open_dataset(f)
    in_data.append(np.stack([ds[v].values for v in in_vars], axis=-1))
    out_data.append(np.stack([ds[v].values for v in out_vars], axis=-1))
    valid_times.append(ds["time"].values)
    run_times.append([run_time] * in_data[-1].shape[0])
    ds.close()

##### Stack the  data into single arrays instead of lists of arrays
This is done to make it easier to feed the data into the ML algorithms

In [ ]:
all_in_data = np.vstack(in_data)
all_out_data = np.vstack(out_data)
all_run_times = np.concatenate(run_times)
all_valid_times = np.concatenate(valid_times)

##### Deallocate the lists of temporary arrays to save memory

In [ ]:
del in_data[:], out_data[:], run_times[:], valid_times[:]
del in_data, out_data, run_times, valid_times

##### Find the maximum vorticity values in the file

In [ ]:
max_vort = all_out_data[:, :, :, 0].max(axis=-1).max(axis=-1)
vort_thresh = 0.008
print(percentileofscore(max_vort, vort_thresh))
vort_labels = np.where(max_vort > vort_thresh, 1, 0)

##### Create some histograms that show the distribution of the data

In [ ]:
plt.figure(figsize=(8, 5))
plt.hist(max_vort, bins=50, cumulative=True, density=True)
plt.plot(np.ones(10) * vort_thresh, np.linspace(0, 1, 10))
plt.yticks(np.arange(0, 1.1, 0.1))
plt.grid()
plt.xlabel("1 km AGL Relative Vorticity")
plt.ylabel("Cumulative Distribution")

In [ ]:
fig, axes = plt.subplots(all_in_data.shape[-1], 1, figsize=(6, all_in_data.shape[-1] * 4))
for a, ax in enumerate(axes):
    ax.hist(all_in_data[:, :, :, a].ravel(), 50)
    ax.set_ylabel(in_vars[a])

##### Plot a storm example using what we've gone over so far

In [ ]:
rot_ex = max_vort.argmax()
plt.figure(figsize=(8, 8))
plt.pcolormesh(all_in_data[rot_ex, :, :, 0], cmap="gist_ncar", vmin=-10, vmax=85)
plt.colorbar()
plt.quiver(all_in_data[rot_ex, :, :, 1], all_in_data[rot_ex, :, :, 2])
plt.contour(all_out_data[rot_ex, :, :, 0])
plt.title("Storm Example {0} Valid ".format(rot_ex) + pd.Timestamp(all_valid_times[rot_ex]).strftime("%Y-%m-%d %H:%M")) 

### Separating into training and test sets 
We need to separate the full data set into two different groups.  The first group is what we feed into the model to train it.  The second group is what we use to test with to see if the model performs as expected.  It's important to create the groups correctly by knowing your data.  For example, is your data time dependant? If so, would randomly assigning data to these groups make it harder for the model to pick up on patterns?  

Picking the correct amount of data to put in each group is equally as important.  Picking the incorrect amount of data (and also picking the incorrect groups) can cause overfitting.  This happens when the model that is generated isn't generalized enough for prediction.

You can try different combinations to see how it effects your model.

##### By date

In [ ]:
split_date = pd.Timestamp("2010-10-28")
train_indices = np.where(all_run_times < split_date)[0]
test_indices = np.where(all_run_times >= split_date)[0]
print ("Size of training set: ",len(train_indices))
print ("Size of test set: ",len(test_indices))

##### By random index

In [ ]:
from random import shuffle

percent_train = .8

indices = range(len(U.coords['p']))
shuffle(indices)

split = int(len(U.coords['p'])*.8)
print ("Splitting on index: ",split)

train_indices = indices[0:split]
test_indices = indices[split:len(U.coords['p'])-1]

print ("Size of training set: ",len(train_indices))
print ("Size of test set: ",len(test_indices))

#print (train_indices)
#print (test_indices)


##### By index

In [ ]:
percent_train = .8

split = int(len(U.coords['p'])*.8)
print ("Splitting on index: ",split)

train_indices = np.array(range(0,split))
test_indices = np.array(range(split, len(U.coords['p'])))
print ("Size of training set: ",len(train_indices))
print ("Size of test set: ",len(test_indices))

##### Normalizing patch data
Normalizing the data allows observational data to be more easily predicted

In [ ]:
from sklearn.preprocessing import MinMaxScaler

U = xf['U10_curr']

U = U.stack(z=('row','col'))
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(U)
ua_norm = scaler.transform(U)

ua_norm.shape

#####  Using prinicpal component analysis to reduce the dimensionality of the different fields

In [ ]:
from sklearn.decomposition import PCA

pc_objs = []
means = []
sds = []

num_comps = 1
num_vars = ua_norm.shape[0]
pc_train_data = np.zeros((train_indices.size, ua_norm.shape[1]), dtype=np.float32)
pc_test_data = np.zeros((test_indices.size, ua_norm.shape[1]), dtype=np.float32)
for v in range(num_vars):
    pc_objs.append(PCA(n_components=num_comps))
    var_data = ua_norm
    pc_train_data[:, v * num_comps: (v + 1) * num_comps] = pc_objs[v].fit_transform(var_data[train_indices])
    pc_test_data[:, v * num_comps: (v + 1) * num_comps] = pc_objs[v].transform(var_data[test_indices])
    del var_data